# NY Warehouse Flow Forecasting

In this notebook, we tried to predict the inbound & outbound traffic from the warehouse, in order to help allocate workforce. Noted that the max capacity of the warehouse is 3500 orders/day as of July 2019, but our predictions are not limited by that.

There are six types of warehouse activities:

Outbound customer order
Outbound items transfer
Outbound purchase order return
Inbound customer return
Inbound items transfer
Inbound purchase order

In [1]:
import pandas as pd
import numpy as np
import queries.utils as utils
from functools import reduce
from matplotlib import pyplot as plt
from datetime import date, timedelta

#### bigquery client & credentials
import os
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\m.young\Documents\BigQueryAPIKey\modata-79d448dbeef0.json"
client = bigquery.Client()

#### fbprophet 
import fbprophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
from fbprophet.plot import add_changepoints_to_plot

pd.plotting.register_matplotlib_converters() ## bugs.. fbprophet & matplotlib

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


## A. IMPORT DATA

### 1. Actuals Data

In [2]:
# BQ query
wh_actuals_query = """
SELECT
     *
FROM adhoc_analytics.project_inven_forecast_actuals_EST
  """

# put query results into pandas df
data_raw_all = client.query(wh_actuals_query).to_dataframe()

# convert date column to datetime data type
data_raw_all['date_est'] = pd.to_datetime(data_raw_all['date_est'])

# sort by date desc
data_raw_all.sort_values(by='date_est', inplace=True)

# reset index numbers after sort
data_raw_all.reset_index(drop=True, inplace=True)

# replace NaN with 0s
data_raw_all.fillna(0, inplace=True)

### 2. FB Prophet Holidays Data

In [4]:
# BQ query
holidays_inputs_query = """
SELECT
     *
FROM adhoc_analytics.project_inven_forecast_fbprophet_events_inputs
  """

# put query results into pandas df
holidays_inputs = client.query(holidays_inputs_query).to_dataframe()

# convert date column to datetime data type
holidays_inputs['ds'] = pd.to_datetime(holidays_inputs['ds'])

In [5]:
co_holidays = holidays_inputs[holidays_inputs['event_type'] == 'Customer Orders']
co_holidays.reset_index(drop=True, inplace=True)

po_holidays = holidays_inputs[holidays_inputs['event_type'] == 'Purchase Orders']
po_holidays.reset_index(drop=True, inplace=True)

cr_holidays = holidays_inputs[holidays_inputs['event_type'] == 'Customer Returns']
cr_holidays.reset_index(drop=True, inplace=True)

## B. Prepare Data

### 1. Prepare Actuals Data for Multiple Models

In [6]:
# Outbound - Customer Orders
co_dat = data_raw_all[['date_est','OUT_CUST_UNITS']]
co_dat = co_dat.rename(columns={'date_est':'ds','OUT_CUST_UNITS':'y'})

# Outbound - Vendor Returns
vr_dat = data_raw_all[['date_est','OUT_VR_UNITS']]
vr_dat = vr_dat.rename(columns={'date_est':'ds','OUT_VR_UNITS':'y'})

# Outbound - Transfer Orders
oto_dat = data_raw_all[['date_est','OUT_TO_UNITS']]
oto_dat = oto_dat.rename(columns={'date_est':'ds','OUT_TO_UNITS':'y'})

# Inbound - Customer Returns
cr_dat = data_raw_all[['date_est','IN_CUST_UNITS']]
cr_dat = cr_dat.rename(columns={'date_est':'ds','IN_CUST_UNITS':'y'})

# Inbound - Purchase Orders
po_dat = data_raw_all[['date_est','IN_PO_UNITS']]
po_dat = po_dat.rename(columns={'date_est':'ds','IN_PO_UNITS':'y'})

# Inbound - Transfer Orders
ito_dat = data_raw_all[['date_est','IN_TO_UNITS']]
ito_dat = ito_dat.rename(columns={'date_est':'ds','IN_TO_UNITS':'y'})

### 2. Prepare Train vs Test Datasets for FB Prophet Models

In [7]:
# Parameters Settings
# Setting the train test cutoff date
split_date = date.today() - timedelta(3)
split_date = split_date.strftime('%Y-%m-%d') 

# Days ahead to forecast
forward_days = 100  # set up how

In [8]:
# Outbound Customer Orders
co_dat_train = co_dat.loc[co_dat.ds<split_date , :]
co_dat_test = co_dat.loc[co_dat.ds>=split_date , :]

# Outbound - Vendor Returns
vr_dat_train = vr_dat.loc[vr_dat.ds<split_date , :]
vr_dat_test = vr_dat.loc[vr_dat.ds>=split_date , :]

# Outbound - Transfer Orders
oto_dat_train = oto_dat.loc[oto_dat.ds<split_date , :]
oto_dat_test = oto_dat.loc[oto_dat.ds>=split_date , :]

# Inbound - Customer Returns
cr_dat_train = cr_dat.loc[cr_dat.ds<split_date , :]
cr_dat_test = cr_dat.loc[cr_dat.ds>=split_date , :]

# Inbound - Purchase Orders
po_dat_train = po_dat.loc[po_dat.ds<split_date , :]
po_dat_test = po_dat.loc[po_dat.ds>=split_date , :]

# Inbound - Transfer Orders
ito_dat_train = ito_dat.loc[ito_dat.ds<split_date , :]
ito_dat_test = ito_dat.loc[ito_dat.ds>=split_date , :]


In [9]:
# Outbound Customer Orders
co_dat_all = pd.concat([co_dat_train, co_dat_test])

# Outbound - Vendor Returns
vr_dat_all = pd.concat([vr_dat_train, vr_dat_test])

# Outbound - Transfer Orders
oto_dat_all = pd.concat([oto_dat_train, oto_dat_test])

# Inbound - Customer Returns
cr_dat_all = pd.concat([cr_dat_train, cr_dat_test])

# Inbound - Purchase Orders
po_dat_all = pd.concat([po_dat_train, po_dat_test])

# Inbound - Transfer Orders
ito_dat_all = pd.concat([ito_dat_train, ito_dat_test])

In [10]:
fbprophet.__version__
Prophet = fbprophet.Prophet

## C. FB Prophet Model - Outbound Customer Orders

In [11]:
co_m = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           ,holidays = co_holidays
           )

co_m_full = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           ,holidays = co_holidays
           )

C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\fbprophet\forecaster.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.holidays['ds'] = pd.to_datetime(self.holidays['ds'])


In [12]:
# fit the model
co_m.fit(co_dat_train)
co_m_full.fit(co_dat_all)

# predict forward_days 
co_future = co_m.make_future_dataframe(periods=len(co_dat_test)+forward_days, freq='1D')
co_forecast = co_m.predict(co_future)

C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


## D. FB Prophet Model - Inbound Purchase Orders

In [13]:
# ADDITONAL REGRESSOR - Purchase Orders Estimated Delivery Windows

# BQ query
po_est_query = """
SELECT
     date AS ds
    ,IN_PO_EST AS reg
FROM adhoc_analytics.project_inven_forecast_purchase_orders
  """

# put query results into pandas df
po_reg_1_full = client.query(po_est_query).to_dataframe()

# convert date column to datetime data type
po_reg_1_full['ds'] = pd.to_datetime(po_reg_1_full['ds'])

# sort by date desc
po_reg_1_full.sort_values(by='ds', inplace=True)

# reset index numbers after sort
po_reg_1_full.reset_index(drop=True, inplace=True)

# replace NaN with 0s
po_reg_1_full.fillna(0, inplace=True)

# split for train test
po_reg_1_train = po_reg_1_full[po_reg_1_full['ds']<split_date]

In [14]:
po_m = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           ,holidays = po_holidays
           )

po_m_full = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           ,holidays = po_holidays
           )

# ### Add extra regressors
po_dat_train['reg'] = po_reg_1_train['reg']
po_m.add_regressor('reg', prior_scale = 10, mode='multiplicative') 

po_dat_all['reg'] = po_reg_1_full['reg']
po_m_full.add_regressor('reg', prior_scale = 10, mode='multiplicative') 

C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\fbprophet\forecaster.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.holidays['ds'] = pd.to_datetime(self.holidays['ds'])
C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
# fit the model
po_m.fit(po_dat_train)
po_m_full.fit(po_dat_all)

# predict forward_days 
po_future = po_m.make_future_dataframe(periods=len(po_dat_test)+forward_days, freq='1D')
po_future['reg'] = po_reg_1_full['reg']

po_forecast = po_m.predict(po_future)

C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


## E. FB Prophet Model - Inbound Customer Returns

In [16]:
cr_m = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
            ,holidays = cr_holidays
            
           )

cr_m_full = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
            ,holidays = cr_holidays
           )

# fit the model
cr_m.fit(cr_dat_train)
cr_m_full.fit(cr_dat_all)

# predict forward_days 
cr_future = cr_m.make_future_dataframe(periods=len(cr_dat_test)+forward_days, freq='1D')
cr_forecast = cr_m.predict(cr_future)

C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\fbprophet\forecaster.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.holidays['ds'] = pd.to_datetime(self.holidays['ds'])
C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype

## F. FB Prophet Model - Inbound Transfer Orders

In [17]:
ito_m = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           #,holidays = ito_holidays
           )

ito_m_full = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           #,holidays = ito_holidays
           )

# fit the model
ito_m.fit(ito_dat_train)
ito_m_full.fit(ito_dat_all)

# predict forward_days 
ito_future = ito_m.make_future_dataframe(periods=len(ito_dat_test)+forward_days, freq='1D')
ito_forecast = ito_m.predict(ito_future)

C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


## G. FB Prophet Model - Outbound Vendor Returns

In [18]:
vr_m = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           #,holidays = vr_holidays
           )

vr_m_full = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           #,holidays = vr_holidays
           )

# fit the model
vr_m.fit(vr_dat_train)
vr_m_full.fit(vr_dat_all)

# predict forward_days 
vr_future = vr_m.make_future_dataframe(periods=len(vr_dat_test)+forward_days, freq='1D')
vr_forecast = vr_m.predict(vr_future)

C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


## H. FB Prophet Model - Outbound Transfer Orders

In [19]:
oto_m = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           #,holidays = oto_holidays
           )

oto_m_full = Prophet(changepoint_prior_scale=0.1, changepoint_range=0.9 ,\
            seasonality_mode='multiplicative', seasonality_prior_scale = 10 ,\
            holidays_prior_scale = 10, \
            yearly_seasonality=True, \
            weekly_seasonality=True, \
            daily_seasonality=False \
           #,holidays = oto_holidays
           )

# fit the model
oto_m.fit(oto_dat_train)
oto_m_full.fit(oto_dat_all)

# predict forward_days 
oto_future = oto_m.make_future_dataframe(periods=len(oto_dat_test)+forward_days, freq='1D')
oto_forecast = oto_m.predict(oto_future)

C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


# --FINAL FB Prophet Ensemble Outputs--

In [20]:
co_forecast.loc[ co_forecast['yhat'] < 0.0, 'yhat'] = 0
po_forecast.loc[ po_forecast['yhat'] < 0.0, 'yhat'] = 0
cr_forecast.loc[ cr_forecast['yhat'] < 0.0, 'yhat'] = 0
ito_forecast.loc[ito_forecast['yhat'] < 0.0, 'yhat'] = 0
oto_forecast.loc[oto_forecast['yhat'] < 0.0, 'yhat'] = 0
vr_forecast.loc[ vr_forecast['yhat'] < 0.0, 'yhat'] = 0

co_ensemble  =  co_forecast[['ds','yhat']]
po_ensemble  =  po_forecast[['ds','yhat']]
cr_ensemble  =  cr_forecast[['ds','yhat']]
ito_ensemble = ito_forecast[['ds','yhat']]
oto_ensemble = oto_forecast[['ds','yhat']]
vr_ensemble  =  vr_forecast[['ds','yhat']]

co_ensemble.rename(columns  ={'ds':'date_est','yhat':'fcst_out_co'} ,inplace=True)  
po_ensemble.rename(columns  ={'yhat':'fcst_in_po'} ,inplace=True)   
cr_ensemble.rename(columns  ={'yhat':'fcst_in_cr'} ,inplace=True)   
ito_ensemble.rename(columns ={'yhat':'fcst_in_ito'} ,inplace=True)  
oto_ensemble.rename(columns ={'yhat':'fcst_out_oto'} ,inplace=True)  
vr_ensemble.rename(columns  ={'yhat':'fcst_out_vr'} ,inplace=True)   

C:\Users\m.young\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [21]:
# join outputs

dfs = [co_ensemble[['date_est','fcst_out_co']] \
      ,po_ensemble[['ds','fcst_in_po']] \
      ,cr_ensemble[['ds','fcst_in_cr']] \
      ,ito_ensemble[['ds','fcst_in_ito']] \
      ,oto_ensemble[['ds','fcst_out_oto']] \
      ,vr_ensemble[['ds','fcst_out_vr']]] 
nan_value = 0

fcst_ensemble = pd.concat(dfs, join='outer', axis=1).fillna(nan_value)
fcst_ensemble.drop(columns=['ds'], inplace = True)


#create total columns
fcst_ensemble['fcst_total_ins'] =    fcst_ensemble['fcst_in_cr'] \
                                   + fcst_ensemble['fcst_in_po'] \
                                   + fcst_ensemble['fcst_in_ito']

fcst_ensemble['fcst_total_outs'] =   fcst_ensemble['fcst_out_co'] \
                                   + fcst_ensemble['fcst_out_oto'] \
                                   + fcst_ensemble['fcst_out_vr'] \

fcst_ensemble['fcst_total_flow'] =   fcst_ensemble['fcst_out_co'] \
                                   + fcst_ensemble['fcst_out_oto'] \
                                   + fcst_ensemble['fcst_out_vr'] \
                                   + fcst_ensemble['fcst_in_cr'] \
                                   + fcst_ensemble['fcst_in_po'] \
                                   + fcst_ensemble['fcst_in_ito']

fcst_ensemble['fcst_total_flow_adj'] = fcst_ensemble['fcst_out_co'] \
                                   + fcst_ensemble['fcst_out_oto'] \
                                   + fcst_ensemble['fcst_in_cr'] \
                                   + fcst_ensemble['fcst_in_po'] \
                                   + fcst_ensemble['fcst_in_ito'] 

In [22]:
# export model outputs data to bigquery
fcst_ensemble.to_gbq('adhoc_analytics.project_inven_forecast_fbprophet_outputs',project_id='moda-operandi-dw',if_exists = 'replace')

1442 out of 1442 rows loaded._gbq.gbq:
1it [00:03,  3.34s/it]


In [23]:
# BQ query | model outputs with actuals
fbprophet_outputs_query = """
SELECT
     CURRENT_TIMESTAMP()              AS MODEL_UPDATED_TIMESTAMP
    ,timestamp(CAST(fct.date_est AS DATETIME),"America/New_York")   AS DATE_EST
    ,ROUND(fcst_out_co,0)	     AS FORECAST_out_customer_order_units
    ,ROUND(fcst_out_oto,0)	   AS FORECAST_out_transfer_order_units
    ,ROUND(fcst_out_vr,0)	     AS FORECAST_out_vendor_return_units
    ,ROUND(fcst_in_cr,0)	     AS FORECAST_in_customer_return_units
    ,ROUND(fcst_in_ito,0)	     AS FORECAST_in_transfer_order_units
    ,ROUND(fcst_in_po,0)	     AS FORECAST_in_purchase_order_units
    ,ROUND(fcst_total_outs,0)	 AS FORECAST_total_out_units          
    ,ROUND(fcst_total_ins,0)	 AS FORECAST_total_in_units            
    ,ROUND(fcst_total_flow,0)	 AS FORECAST_total_all_units          

    ,EST.OUT_CUST_UNITS	       AS ACTUALS_out_customer_order_units
    ,EST.OUT_TO_UNITS	         AS ACTUALS_out_transfer_order_units
    ,EST.OUT_VR_UNITS	         AS ACTUALS_out_vendor_return_units
    ,EST.IN_CUST_UNITS	       AS ACTUALS_in_customer_return_units
    ,EST.IN_TO_UNITS	         AS ACTUALS_in_transfer_order_units
    ,EST.IN_PO_UNITS	         AS ACTUALS_in_purchase_order_units  
    ,EST.TTL_OUT_UNITS	       AS ACTUALS_total_out_units          
    ,EST.TTL_IN_UNITS	         AS ACTUALS_total_in_units           
    ,EST.TTL_ALL_UNITS	       AS ACTUALS_total_all_units          
    
FROM adhoc_analytics.project_inven_forecast_fbprophet_outputs fct
     LEFT JOIN adhoc_analytics.project_inven_forecast_actuals_EST EST
            ON CAST(fct.date_est AS DATE) = CAST(EST.date_est AS DATE)
WHERE
     CAST(fct.date_est AS DATE) <= DATE_ADD(CURRENT_DATE(), INTERVAL 100 DAY)
     
ORDER BY fct.date_est DESC 
  """

# put query results into pandas df
fbprophet_outputs = client.query(fbprophet_outputs_query).to_dataframe()

In [25]:
# export model outputs with actuals data to bigquery
fbprophet_outputs.to_gbq('adhoc_analytics.project_inven_forecast_fbprophet_outputs_looker',project_id='moda-operandi-dw',if_exists = 'append')

1442 out of 1442 rows loaded._gbq.gbq:
1it [00:04,  4.31s/it]
